In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import logging
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [3]:
# Constants
RELATIONSHIP_TYPES = ['ss', 'bb', 'ms', 'fs', 'fd', 'md', 'sibs', 'gfgs', 'gmgs', 'gfgd', 'gmgd']
RELATIONSHIP_TO_IDX = {rel: idx for idx, rel in enumerate(RELATIONSHIP_TYPES)}
INPUT_SHAPE = (112, 112, 3)  # Correct image size
BATCH_SIZE = 32

# Age difference expectations for relationships (in years)
AGE_EXPECTATIONS = {
    'ss': 5,     # Allow some age difference for sisters
    'bb': 5,     # Allow some age difference for brothers
    'ms': 25,    # Mother-son age difference
    'fs': 30,    # Father-son age difference
    'fd': 30,    # Father-daughter age difference
    'md': 25,    # Mother-daughter age difference
    'sibs': 5,   # Siblings age difference
    'gfgs': 55,  # Grandfather-grandson age difference
    'gmgs': 50,  # Grandmother-grandson age difference
    'gfgd': 55,  # Grandfather-granddaughter age difference
    'gmgd': 50   # Grandmother-granddaughter age difference
}


In [4]:
class KinshipFeatureExtractor(layers.Layer):
    def __init__(self, name="feature_extractor"):
        super(KinshipFeatureExtractor, self).__init__(name=name)
        
        # Initialize ResNet50 with correct input shape
        base_model = tf.keras.applications.ResNet50V2(
            include_top=False,
            weights='imagenet',
            input_shape=INPUT_SHAPE
        )
        
        # Freeze early layers
        for layer in base_model.layers[:100]:
            layer.trainable = False
            
        self.base_model = base_model
        
        # Additional layers
        self.global_pool = layers.GlobalAveragePooling2D()
        self.bn1 = layers.BatchNormalization()
        self.dropout1 = layers.Dropout(0.5)
        self.dense1 = layers.Dense(512, activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.dropout2 = layers.Dropout(0.3)
        self.dense2 = layers.Dense(256)

    def call(self, x, training=False):
        x = self.base_model(x, training=training)
        x = self.global_pool(x)
        x = self.bn1(x, training=training)
        x = self.dropout1(x, training=training)
        x = self.dense1(x)
        x = self.bn2(x, training=training)
        x = self.dropout2(x, training=training)
        x = self.dense2(x)
        return tf.nn.l2_normalize(x, axis=1)

In [5]:
class KinshipModel(Model):
    def __init__(self, margin=0.3, name="kinship_model"):
        super(KinshipModel, self).__init__(name=name)
        self.margin = margin
        self.feature_extractor = KinshipFeatureExtractor()
        
        # Relationship classifier
        self.classifier = tf.keras.Sequential([
            layers.Dense(128, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            layers.Dense(len(RELATIONSHIP_TYPES))
        ])

    def call(self, inputs, training=False):
        anchor, positive, negative = inputs
        
        # Get embeddings
        anchor_embedding = self.feature_extractor(anchor, training=training)
        positive_embedding = self.feature_extractor(positive, training=training)
        negative_embedding = self.feature_extractor(negative, training=training)
        
        # Compute distances
        positive_distance = tf.reduce_sum(tf.square(anchor_embedding - positive_embedding), axis=1)
        negative_distance = tf.reduce_sum(tf.square(anchor_embedding - negative_embedding), axis=1)
        
        # Classification logits from positive pair
        combined_features = tf.concat([anchor_embedding, positive_embedding], axis=1)
        classification_output = self.classifier(combined_features, training=training)
        
        return positive_distance, negative_distance, classification_output

In [6]:

def combined_loss(y_true, y_pred):
    """Combined triplet and classification loss"""
    positive_distance, negative_distance, classification_output = y_pred
    relationship_labels = y_true
    
    # Triplet loss
    margin = 0.3
    triplet_loss = tf.maximum(0., positive_distance - negative_distance + margin)
    
    # Classification loss
    classification_loss = tf.keras.losses.sparse_categorical_crossentropy(
        relationship_labels, 
        tf.nn.softmax(classification_output), 
        from_logits=False
    )
    
    # Combine losses (weighted sum)
    total_loss = triplet_loss + 0.5 * classification_loss
    
    return tf.reduce_mean(total_loss)

In [7]:
def load_and_preprocess_image(image_path):
    """Load and preprocess image"""
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    # No need to resize as images are already 112x112
    image = tf.cast(image, tf.float32) / 255.0  # Simple normalization
    return image

def create_dataset(df, batch_size=BATCH_SIZE):
    """Create dataset from DataFrame containing triplets"""
    def generator():
        for _, row in df.iterrows():
            # Load and preprocess images
            anchor = load_and_preprocess_image(row['Anchor'])
            positive = load_and_preprocess_image(row['Positive'])
            negative = load_and_preprocess_image(row['Negative'])
            
            # Get label (relationship type)
            label = RELATIONSHIP_TO_IDX[row['ptype']]
            
            yield (anchor, positive, negative), label
    
    return tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            (
                tf.TensorSpec(shape=INPUT_SHAPE, dtype=tf.float32),
                tf.TensorSpec(shape=INPUT_SHAPE, dtype=tf.float32),
                tf.TensorSpec(shape=INPUT_SHAPE, dtype=tf.float32)
            ),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [8]:
class KinshipTrainer:
    def __init__(self, model, train_dataset, val_dataset, test_dataset):
        self.model = model
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.test_dataset = test_dataset
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
        
        # Initialize metrics
        self.reset_metrics()
        
        # For tracking history
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'train_accuracy': [],
            'val_accuracy': []
        }
    
    def reset_metrics(self):
        """Initialize/Reset metrics"""
        self.train_loss_values = []
        self.val_loss_values = []
        self.train_accuracy_values = []
        self.val_accuracy_values = []
        
        self.train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        self.val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    
    @tf.function
    def train_step(self, images, labels):
        with tf.GradientTape() as tape:
            predictions = self.model(images, training=True)
            loss = combined_loss(labels, predictions)
        
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        
        # Update accuracy
        self.train_accuracy.update_state(labels, tf.nn.softmax(predictions[2]))
        
        return loss
    
    @tf.function
    def val_step(self, images, labels):
        predictions = self.model(images, training=False)
        loss = combined_loss(labels, predictions)
        
        # Update accuracy
        self.val_accuracy.update_state(labels, tf.nn.softmax(predictions[2]))
        
        return loss
    
    def train(self, epochs=50, early_stopping_patience=5):
        best_val_loss = float('inf')
        patience_counter = 0
        
        for epoch in range(epochs):
            # Reset metrics for this epoch
            self.reset_metrics()
            
            # Training loop
            train_losses = []
            with tqdm(self.train_dataset, desc=f"Epoch {epoch + 1}/{epochs}") as pbar:
                for images, labels in pbar:
                    loss = self.train_step(images, labels)
                    train_losses.append(float(loss))
                    
                    # Update progress bar
                    pbar.set_postfix({
                        'loss': f'{np.mean(train_losses):.4f}',
                        'accuracy': f'{self.train_accuracy.result().numpy():.4f}'
                    })
            
            # Calculate average training loss
            avg_train_loss = np.mean(train_losses)
            
            # Validation loop
            val_losses = []
            for images, labels in self.val_dataset:
                val_loss = self.val_step(images, labels)
                val_losses.append(float(val_loss))
            
            # Calculate average validation loss
            avg_val_loss = np.mean(val_losses)
            
            # Get accuracies
            train_acc = self.train_accuracy.result().numpy()
            val_acc = self.val_accuracy.result().numpy()
            
            # Update history
            self.history['train_loss'].append(avg_train_loss)
            self.history['val_loss'].append(avg_val_loss)
            self.history['train_accuracy'].append(train_acc)
            self.history['val_accuracy'].append(val_acc)
            
            # Print metrics
            print(
                f"\nEpoch {epoch + 1}/{epochs}:\n"
                f"Train Loss: {avg_train_loss:.4f} - Train Accuracy: {train_acc:.4f}\n"
                f"Val Loss: {avg_val_loss:.4f} - Val Accuracy: {val_acc:.4f}"
            )
            
            # Early stopping check
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                self.model.save_weights('best_kinship_model.h5')
                patience_counter = 0
                print("Saved new best model!")
            else:
                patience_counter += 1
                if patience_counter >= early_stopping_patience:
                    print("Early stopping triggered!")
                    break
        
        # Load best weights
        self.model.load_weights('best_kinship_model.h5')
        
        # Plot training history
        self.plot_history()
    
    def plot_history(self):
        """Plot training history"""
        plt.figure(figsize=(12, 4))
        
        # Plot loss
        plt.subplot(1, 2, 1)
        plt.plot(self.history['train_loss'], label='Train Loss')
        plt.plot(self.history['val_loss'], label='Val Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        
        # Plot accuracy
        plt.subplot(1, 2, 2)
        plt.plot(self.history['train_accuracy'], label='Train Accuracy')
        plt.plot(self.history['val_accuracy'], label='Val Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig('training_history.png')
        plt.show()
    
    def evaluate(self):
        """Evaluate model on test set"""
        test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        all_losses = []
        all_labels = []
        all_predictions = []
        
        for images, labels in tqdm(self.test_dataset, desc="Evaluating"):
            predictions = self.model(images, training=False)
            loss = combined_loss(labels, predictions)
            all_losses.append(float(loss))
            
            classification_logits = predictions[2]
            test_accuracy.update_state(labels, tf.nn.softmax(classification_logits))
            
            all_labels.extend(labels.numpy())
            all_predictions.extend(tf.argmax(classification_logits, axis=1).numpy())
        
        # Print metrics
        print(f"\nTest Results:")
        print(f"Test Loss: {np.mean(all_losses):.4f}")
        print(f"Test Accuracy: {test_accuracy.result().numpy():.4f}")
        
        # Print classification report
        print("\nClassification Report:")
        print(classification_report(all_labels, all_predictions, 
                                 target_names=RELATIONSHIP_TYPES))
        
        # Plot confusion matrix
        cm = confusion_matrix(all_labels, all_predictions)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=RELATIONSHIP_TYPES,
                   yticklabels=RELATIONSHIP_TYPES)
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('confusion_matrix.png')
        plt.show()

In [9]:
# Data loading and preparation
print("Loading dataset...")
triplets_df = pd.read_csv('../data/processed/fiw/train/filtered_triplets_with_labels.csv')

# Split dataset
print("Splitting dataset...")
train_df, temp_df = train_test_split(
    triplets_df, test_size=0.3, stratify=triplets_df['ptype'], random_state=42
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.5, stratify=temp_df['ptype'], random_state=42
)

print(f"\nDataset splits:")
print(f"Train: {len(train_df)} samples")
print(f"Validation: {len(val_df)} samples")
print(f"Test: {len(test_df)} samples")

Loading dataset...
Splitting dataset...

Dataset splits:
Train: 132685 samples
Validation: 28432 samples
Test: 28433 samples


In [10]:
# Create datasets
print("\nCreating datasets...")
train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)
test_dataset = create_dataset(test_df)


Creating datasets...


2024-10-27 19:32:34.337687: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-10-27 19:32:34.337725: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-10-27 19:32:34.337735: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-10-27 19:32:34.337755: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-27 19:32:34.337770: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
# Create model and trainer
print("\nInitializing model...")
model = KinshipModel()
trainer = KinshipTrainer(model, train_dataset, val_dataset, test_dataset)


Initializing model...


In [12]:
# Train model
print("\nStarting training...")
trainer.train(epochs=50)


Starting training...


Epoch 1/50: 0it [00:00, ?it/s]

: 

In [ ]:
# Evaluate model
trainer.evaluate()